```
$> pip install numpy
$> pip install opencv-python
$> pip install opencv-contrib-python   >> 에러 난다면?
$> pip install --user opencv-contrib-python
```

In [1]:
import cv2
import numpy as np
import sys

In [2]:
video_path = 'data/itzy.mp4'
cap = cv2.VideoCapture(video_path)    # 해당 경로의 동영상을 로드한다.

# 동영상 내보내기 
output_size = (150, 190)    # 출력 사이즈 지정 (너비, 높이)   ->  오류가 나면 output_size를 반으로 줄여준다. !! 

# initialize writing video  ( 복붙 )
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('%s_output.mp4' % (video_path.split('.')[0]), fourcc, cap.get(cv2.CAP_PROP_FPS), output_size)

In [5]:
if not cap.isOpened():       # 동영상이 제대로 로드되면 True를 반환한다.
    sys.exit()
    
# OpneCV의 Object Tracker(총 7개) - 그 중 가장 성능이 좋은 CSRT 사용
OPENCV_OBJECT_TRACKERS = {
"csrt": cv2.TrackerCSRT_create,      # More accurate than KCF but slower. 낮은 프레임(fps25)에서 사용
"kcf": cv2.TrackerKCF_create,        # Fast and accurate
"boosting": cv2.TrackerBoosting_create,
"mil": cv2.TrackerMIL_create,
"tld": cv2.TrackerTLD_create,
"medianflow": cv2.TrackerMedianFlow_create,
"mosse": cv2.TrackerMOSSE_create     # Extremely fast but not as accurate as either KCF or CSRT
}
tracker = OPENCV_OBJECT_TRACKERS['csrt']()
# tracker = cv2.TrackerCSRT_create()   # csrt 오브젝트 트래커를 선언 
    
## ROI(Region of Interest) : 관심영역
# 첫번째 프레임 읽기 (첫번째 프레임에서 ROI 설정)
ret, frame = cap.read()

if not ret:
    print('Failed to read video')
    sys.exit(1)
    
cv2.namedWindow('Select Window')    # ROI 세팅하는 윈도우 이름
cv2.imshow('Select Window', frame)    # cv2.imshow(이미지 창 이름, 파일 명)    # 첫번째 frame 띄워라.
    
# Setting ROI
rect = cv2.selectROI('Select Window', frame, fromCenter=False, showCrosshair=True)  # frame 에서 ROI를 설정하여 rect(사각형)으로 반환한다.
cv2.destroyWindow('Select Window')   # ROI를 선택하고 나면 Window창을 닫아라.

# ROI 세팅후에, tracker 초기화
tracker.init(frame, rect)    # 오브젝트 트래커가 frame와 rect를 따라가게끔 설정한다.

while True:
    ret, frame = cap.read()    # 동영상을 1프레임씩 읽어온다. -> frame 변수에 넣는다.
    
    if not ret:   # 비디오가 끝나면 ret = False
        exit()
        
    ## Start timer
    timer = cv2.getTickCount()
        
    # Update tracker
    success, box = tracker.update(frame)    # frame에서 rect로 설정한 이미지와 비슷한 물체의 위치를 찾아(Update) 반환한다.
                                            # 성공하면 success,   track한 결과를 box에 담는다.
    
    ## Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    
    # box의 왼쪽, 위쪽, 너비, 높이
    left, top, w, h = [int(v) for v in box]    
    
    # 동영상 저장 관련 크기 설정
    # ROI박스 가운데 지점 
    center_x = left + (w / 2)
    center_y = top + (h / 2)
    
    # 결과물 박스(box)
    result_top = int( center_y - (output_size[1] / 2) )
    result_bottom = int( center_y + (output_size[1] / 2) )
    result_left = int( center_x - (output_size[0] / 2) )
    result_right = int( center_x + (output_size[0] / 2) )
    
    # crop image (결과 이미지)
    result_img = frame[result_top:result_bottom, result_left:result_right].copy()    # .copy()를 해주어야 결과화면에 흰색 네모 나오지x
    
    # 이미지(img)에 사각형을 그린다.
    cv2.rectangle(frame, pt1=(left, top), pt2=(left + w, top + h), color=(255, 255, 255),
                 thickness=3)   
                                # pt1 : 왼쪽 위,   pt2: 오른쪽 아래
                                # color: 사각형의 둘레 색깔
                                # thickness : 사각형의 둘레 두께
    
    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
    
    cv2.imshow('main_img', frame)
    cv2.imshow('result_img', result_img)   # 결과물 박스 띄우기
    
    # 이미지를 동영상으로 저장한다.
    out.write(result_img)
    
    cv2.waitKey(1)          # 이게 없으면 윈도우가 자동으로 꺼진다.
    if cv2.waitKey(1) == ord('q'):      # q 누르면 작동중지, 종료
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        break
        

## How to Use ?
```
$> ObjectTracking.py
1. 마우스를 드래그해서, 목표 target 박스 설정
2. SpaceBar 누르기
```